In [1]:
import pandas as pd
import numpy as np
import awkward as ak

#### Load data from csv files generated from simulation with address swapping (TOP) and without

In [2]:
inputDirWithSwap="UVM_r2g_Layer5_TOP"
inputDirBeforeSwap="UVM_r2g_Layer5"

commentChar='#'

def toHex(x,N=4):
    return format(x, f'0{N}x')
toHex = np.vectorize(toHex)

columns = [f'FRAMEQ_{i}' for i in range(26)]

#in df, load data from r2g, not at top level                                                                                                                                                                                            
df = pd.read_csv(f'{inputDirBeforeSwap}/Formatter_Buffer_Output_FrameQ.csv', skipinitialspace=True,comment=commentChar)
df.columns = columns
df[columns] = toHex(df)
df['FRAMEQ_NUMW'] = pd.read_csv(f'{inputDirBeforeSwap}/Formatter_Buffer_Output_FrameQ_NumW.csv', skipinitialspace=True,comment=commentChar)

#load data from top level r2g, which has swapped address map between algorithm and formatter                                                                                                                                            
df_Swapped = pd.read_csv(f'{inputDirWithSwap}/Formatter_Buffer_Output_FrameQ.csv', skipinitialspace=True,comment=commentChar)
df_Swapped.columns = [f'FRAMEQ_{i}' for i in range(26)]
df_Swapped[columns] = toHex(df_Swapped)
df_Swapped['FRAMEQ_NUMW'] = pd.read_csv(f'{inputDirWithSwap}/Formatter_Buffer_Output_FrameQ_NumW.csv', skipinitialspace=True,comment=commentChar)


In [3]:
def unpack(row):
    #unpack header information from first 16 bit word
    headerWord = int(row['FRAMEQ_0'],16)
    bx = (headerWord>>11)
    packetType = (headerWord>>8) & 7
    modSum = headerWord & 255

    if packetType==4:
        data = row[[f'FRAMEQ_{i}' for i in range(26)]]
        N = int(row['FRAMEQ_NUMW'])
        data = format(int(''.join(data[1:N]),16),f'0{(N-1)*16}b')

        #get 48 bits of address map                                                                                                                                                                                                     
        addrMap = np.array([int(x) for x in data[:48]])
        data = data[48:]

        #convert into decimal values of addresses                                                                                                                                                                                       
        addresses = np.arange(48)[addrMap==1]

        #get list of charges, 7 bits at a time                                                                                                                                                                                          
        charges = np.array([int(data[x:x+7],2) for x in range(0,len(data),7)])

    elif packetType==2:
        data = row[[f'FRAMEQ_{i}' for i in range(26)]]
        N = int(row['FRAMEQ_NUMW'])
        data = format(int(''.join(data[1:N]),16),f'0{(N-1)*16}b')

        NTC = int(data[:3],2)
        data = data[3:]

        #get list of addresses, 6 bits at a time                                                                                                                                                                                        
        addresses = np.array([int(data[x:x+6],2)for x in range(0,NTC*6,6)])
        data = data[NTC*6:]

        #get list of charges, 7 bits at a time                                                                                                                                                                                          
        charges = np.array([int(data[x:x+7],2)for x in range(0,NTC*7,7)])

    else:
        charges = np.array([])
        addresses = np.array([])
    return bx, packetType, modSum, addresses, charges


In [4]:
unpackedData         = df.apply(unpack,axis=1).apply(pd.Series,index=['bx','packetType','modSum','addresses','charges'])
unpackedData_Swapped = df_Swapped.apply(unpack,axis=1).apply(pd.Series,index=['bx','packetType','modSum','addresses','charges'])

In [5]:
unpackedData

,bx,packetType,modSum,addresses,charges
0,3,4,100,"[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47]","[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 7..."
1,4,4,132,"[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47]","[104, 104, 104, 104, 104, 104, 104, 104, 104, ..."
2,5,0,36,[],[]
3,6,4,69,"[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47]","[40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 4..."
4,7,4,100,"[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47]","[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 7..."
...,...,...,...,...,...
10687,5,4,85,"[1, 4, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[32, 46, 43, 39, 33, 42, 48, 44, 38, 45, 42, 4..."
10688,6,4,80,"[0, 1, 2, 3, 6, 8, 9, 12, 15, 19, 20, 21, 22, ...","[32, 42, 40, 37, 31, 33, 47, 38, 43, 33, 40, 3..."
10689,7,4,79,"[0, 2, 5, 10, 16, 18, 19, 21, 22, 24, 25, 27, ...","[43, 36, 33, 37, 46, 42, 39, 32, 34, 47, 41, 3..."
10690,8,4,80,"[0, 3, 4, 7, 9, 10, 12, 14, 15, 16, 17, 19, 20...","[43, 34, 38, 41, 31, 32, 34, 42, 42, 40, 49, 4..."


In [6]:
unpackedData_Swapped

,bx,packetType,modSum,addresses,charges
0,3,4,100,"[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44]","[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 7..."
1,4,4,132,"[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44]","[104, 104, 104, 104, 104, 104, 104, 104, 104, ..."
2,5,0,36,[],[]
3,6,4,69,"[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44]","[40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 4..."
4,7,4,100,"[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44]","[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 7..."
...,...,...,...,...,...
10687,5,4,85,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[32, 46, 43, 39, 33, 42, 48, 44, 38, 45, 42, 4..."
10688,6,4,80,"[0, 1, 2, 4, 6, 9, 12, 13, 14, 15, 16, 17, 18,...","[32, 42, 40, 37, 31, 33, 47, 38, 43, 33, 40, 3..."
10689,7,4,79,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 15, 19, 20, 22...","[43, 36, 33, 37, 46, 42, 39, 32, 34, 47, 41, 3..."
10690,8,4,80,"[0, 1, 2, 4, 10, 11, 19, 20, 22, 24, 25, 26, 2...","[43, 34, 38, 41, 31, 32, 34, 42, 42, 40, 49, 4..."


#### Grab charge and address values, store in awkward arrays for easy comparisons

In [7]:
charges_Correct = ak.Array(unpackedData.charges.values)
charges_TopLevel = ak.Array(unpackedData_Swapped.charges.values)

addresses_Correct = ak.Array(unpackedData.addresses.values)
addresses_TopLevel = ak.Array(unpackedData_Swapped.addresses.values)


In [8]:
print('Correct Charge Values')
print(charges_Correct)
print()

print('Charge Values from Top Level Sim')
print(charges_TopLevel)
print()

print('Compare all charges')
print('    Agree =', ak.all(charges_Correct==charges_TopLevel))


Correct Charge Values
[[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, ... 37, 41, 37, 38, 35, 34, 33, 33, 0, 0]]

Charge Values from Top Level Sim
[[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, ... 37, 41, 37, 38, 35, 34, 33, 33, 0, 0]]

Compare all charges
    Agree = True


In [9]:
print('Correct Address Values')
print(addresses_Correct)
print()

print('Address Values from Top Level Sim')
print(addresses_TopLevel)
print()

print('Compare all addresses (before inverting, expected to fail)')
print('    Agree =', ak.all(addresses_Correct==addresses_TopLevel))


Correct Address Values
[[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, ... 27, 28, 29, 33, 35, 36, 38, 39, 42, 46]]

Address Values from Top Level Sim
[[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, ... 27, 28, 33, 37, 39, 40, 41, 43, 44, 45]]

Compare all addresses (before inverting, expected to fail)
    Agree = False


#### Invert the address lists from top level.  
Reverse the order of the list and subtract from 47

In [10]:
addresses_TopLevel_Inverted = 47 - addresses_TopLevel[:,::-1]

In [11]:
print('Address Values from Top Level Sim after inverting')
print(addresses_TopLevel_Inverted)
print()

print('Compare all addresses (after inverting)')
print('    Agree =', ak.all(addresses_Correct==addresses_TopLevel_Inverted))


Address Values from Top Level Sim after inverting
[[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, ... 27, 28, 29, 33, 35, 36, 38, 39, 42, 46]]

Compare all addresses (after inverting)
    Agree = True
